In [1]:
import pandas as pd
pd.set_option("display.max_rows", 400)  # enough for 365
pd.set_option("display.max_columns", None)  # show all cols if needed

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/09 15:07:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
!ls -lhR data/raw/antwerp

data/raw/antwerp:
total 16K
drwxrwxr-x 2 sweantu sweantu 4.0K Sep  7 04:56 2024-09-26
drwxrwxr-x 2 sweantu sweantu 4.0K Sep  7 04:57 2024-12-28
drwxrwxr-x 2 sweantu sweantu 4.0K Sep  7 04:57 2025-03-21
drwxrwxr-x 2 sweantu sweantu 4.0K Sep  9 01:34 2025-06-25

data/raw/antwerp/2024-09-26:
total 2.5M
-rw-rw-r-- 1 sweantu sweantu 2.5M Sep 30  2024 calendar.csv.gz

data/raw/antwerp/2024-12-28:
total 2.5M
-rw-rw-r-- 1 sweantu sweantu 2.5M Jan  6  2025 calendar.csv.gz

data/raw/antwerp/2025-03-21:
total 2.5M
-rw-rw-r-- 1 sweantu sweantu 2.5M Mar 23 19:50 calendar.csv.gz

data/raw/antwerp/2025-06-25:
total 3.4M
-rw-rw-r-- 1 sweantu sweantu 2.2M Jun 30 16:21 calendar.csv.gz
-rw-rw-r-- 1 sweantu sweantu 1.2M Jun 30 16:20 listings.csv.gz


In [11]:
df_reviews_pd = pd.read_csv("data/raw/sunshine-coast/2025-08-31/reviews.csv.gz", nrows=1000)

In [13]:
df_listings_pd = pd.read_csv("data/raw/sunshine-coast/2025-08-31/listings.csv.gz", nrows=1000)

In [7]:
df_calendar_pd = pd.read_csv("data/raw/antwerp/2025-06-25/calendar.csv.gz", nrows=1000)

In [13]:
calendar_schema = types.StructType([
    types.StructField('listing_id', types.LongType(), True), 
    types.StructField('date', types.StringType(), True), 
    types.StructField('available', types.StringType(), True), 
    types.StructField('price', types.StringType(), True), 
    types.StructField('adjusted_price', types.StringType(), True), 
    types.StructField('minimum_nights', types.LongType(), True), 
    types.StructField('maximum_nights', types.LongType(), True)
])

In [56]:
df_calendar = spark.read \
    .option("header", "true") \
    .option("encoding", "utf-8") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .option("multiLine", "true") \
    .option("mode", "PERMISSIVE") \
    .schema(calendar_schema) \
    .csv("data/raw/antwerp/2024-09-26/calendar.csv.gz")

In [18]:
from pyspark.sql.functions import col

In [20]:
df_calendar.where(col('adjusted_price').isNotNull()).show()

[Stage 2:>                                                          (0 + 1) / 1]

+----------+----+---------+-----+--------------+--------------+--------------+
|listing_id|date|available|price|adjusted_price|minimum_nights|maximum_nights|
+----------+----+---------+-----+--------------+--------------+--------------+
+----------+----+---------+-----+--------------+--------------+--------------+



In [21]:
df_calendar.schema

StructType([StructField('listing_id', LongType(), True), StructField('date', StringType(), True), StructField('available', StringType(), True), StructField('price', StringType(), True), StructField('adjusted_price', StringType(), True), StructField('minimum_nights', LongType(), True), StructField('maximum_nights', LongType(), True)])

In [57]:
special_casts = {
    "price": lambda col: F.regexp_replace(col, "[€,]", "").cast(types.DoubleType()),
    "adjusted_price": lambda col: F.regexp_replace(col, "[€,]", "").cast(types.DoubleType()),
    "date": lambda col: F.to_date(col, "yyyy-MM-dd")
}
for field in df_calendar.schema:
    col_name = field.name
    col_type = field.dataType

    if col_name in special_casts:
        df_calendar = df_calendar.withColumn(col_name, special_casts[col_name](F.col(col_name)))

In [37]:
df_calendar.where(col('adjusted_price').isNotNull()).show()

[Stage 9:>                                                          (0 + 1) / 1]

+----------+----+---------+-----+--------------+--------------+--------------+
|listing_id|date|available|price|adjusted_price|minimum_nights|maximum_nights|
+----------+----+---------+-----+--------------+--------------+--------------+
+----------+----+---------+-----+--------------+--------------+--------------+



In [58]:
df_calendar.createOrReplaceTempView("calendar")

In [60]:
df_calendar_filtered = spark.sql("""
select *
from calendar
where
    date < '2024-12-28'
""")

In [61]:
output_path = f'data/pq/antwerp/2024-09-26/'
df_calendar_filtered.repartition(4).write.parquet(output_path)

In [4]:
!ls -lhR data/pq/antwerp/

data/pq/antwerp/:
total 12K
drwxr-xr-x 2 sweantu sweantu 4.0K Sep  8 01:35 2024-09-26
drwxr-xr-x 2 sweantu sweantu 4.0K Sep  8 01:34 2024-12-28
drwxr-xr-x 2 sweantu sweantu 4.0K Sep  8 01:31 2025-03-21

data/pq/antwerp/2024-09-26:
total 1.1M
-rw-r--r-- 1 sweantu sweantu    0 Sep  8 01:35 _SUCCESS
-rw-r--r-- 1 sweantu sweantu 276K Sep  8 01:35 part-00000-c3f53354-9f93-4283-b6bf-fd5772d19d31-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 276K Sep  8 01:35 part-00001-c3f53354-9f93-4283-b6bf-fd5772d19d31-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 276K Sep  8 01:35 part-00002-c3f53354-9f93-4283-b6bf-fd5772d19d31-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 279K Sep  8 01:35 part-00003-c3f53354-9f93-4283-b6bf-fd5772d19d31-c000.snappy.parquet

data/pq/antwerp/2024-12-28:
total 984K
-rw-r--r-- 1 sweantu sweantu    0 Sep  8 01:34 _SUCCESS
-rw-r--r-- 1 sweantu sweantu 244K Sep  8 01:34 part-00000-91a0addc-e3ae-428f-a68c-f9a0aa1d403e-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 24

In [5]:
df_calendar_pq = spark.read.parquet("data/pq/antwerp/*/*")

In [6]:
df_calendar_pq.createOrReplaceTempView("calendar")

In [13]:
pd.set_option("display.max_rows", 400)  # enough for 365
pd.set_option("display.max_columns", None)  # show all cols if needed

In [14]:
spark.sql("""
select *
from calendar
where listing_id = 50904
order by date
""").toPandas()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,50904,2024-09-26,f,NaN,NaN,1,60
1,50904,2024-09-27,f,NaN,NaN,2,60
2,50904,2024-09-28,f,NaN,NaN,2,60
3,50904,2024-09-29,f,NaN,NaN,1,60
4,50904,2024-09-30,t,NaN,NaN,1,60
5,50904,2024-10-01,t,NaN,NaN,1,60
6,50904,2024-10-02,t,NaN,NaN,1,60
7,50904,2024-10-03,t,NaN,NaN,1,60
8,50904,2024-10-04,f,NaN,NaN,2,60
9,50904,2024-10-05,f,NaN,NaN,2,60


In [16]:
spark.sql("""
with cte as (
    select 
        listing_id, 
        count(*) as total,
        COUNT(*) FILTER (WHERE available = 't') as available_count,
        sum(case when available = 'f' then 1 else 0 end) as unavailable_count
    from calendar
    group by listing_id
)
, cte2 as (
    select 
        *,
        case
            when unavailable_count = 0 then '0'
            when unavailable_count <= 30 then '1-30'
            when unavailable_count <= 60 then '31-60'
            when unavailable_count <= 90 then '61-90'
            when unavailable_count <= 120 then '91-120'
            when unavailable_count <= 150 then '121-150'
            when unavailable_count <= 180 then '151-180'
            when unavailable_count <= 210 then '181-210'
            when unavailable_count <= 240 then '211-240'
            else '241-255+'
        end as occupancy
    from cte
)
select
    occupancy,
    count(*) as total_listings
from cte2
    join listings on cte2.listing_id = listings.id
group by occupancy
""").show()

+---------+--------------+
|occupancy|total_listings|
+---------+--------------+
|        0|            27|
|  121-150|           155|
| 241-255+|           356|
|  181-210|           129|
|   91-120|           316|
|  211-240|            64|
|    61-90|           327|
|     1-30|           372|
|    31-60|           440|
|  151-180|           138|
+---------+--------------+



In [26]:
spark.sql("""
 select 
        listing_id, 
        date, 
        available
    from calendar
    where listing_id = 50904
    order by available, date
""").show()

+----------+----------+---------+
|listing_id|      date|available|
+----------+----------+---------+
|     50904|2024-09-26|        f|
|     50904|2024-09-27|        f|
|     50904|2024-09-28|        f|
|     50904|2024-09-29|        f|
|     50904|2024-10-04|        f|
|     50904|2024-10-05|        f|
|     50904|2024-10-16|        f|
|     50904|2024-10-17|        f|
|     50904|2024-10-18|        f|
|     50904|2024-10-19|        f|
|     50904|2024-10-25|        f|
|     50904|2024-10-26|        f|
|     50904|2024-11-07|        f|
|     50904|2024-11-08|        f|
|     50904|2024-11-09|        f|
|     50904|2024-11-15|        f|
|     50904|2024-11-16|        f|
|     50904|2024-12-05|        f|
|     50904|2024-12-06|        f|
|     50904|2024-12-07|        f|
+----------+----------+---------+
only showing top 20 rows



In [50]:
df_calendar_pq.printSchema()

root
 |-- listing_id: long (nullable = true)
 |-- date: date (nullable = true)
 |-- available: string (nullable = true)
 |-- price: double (nullable = true)
 |-- adjusted_price: double (nullable = true)
 |-- minimum_nights: long (nullable = true)
 |-- maximum_nights: long (nullable = true)



In [8]:
df_listings_pd[['id','scrape_id','last_scraped','name','host_id','host_name','neighbourhood_cleansed','neighbourhood_group_cleansed','latitude','longitude','room_type','price','minimum_nights','maximum_nights','license']].tail(10)

,id,scrape_id,last_scraped,name,host_id,host_name,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,price,minimum_nights,maximum_nights,license
990,1696381,20250801203054,2025-08-02,Charming Room Near Central Park,2717428,Ellen,East Harlem,Manhattan,40.79959,-73.93857,Private room,NaN,30,1125,NaN
991,1702032,20250801203054,2025-08-02,The Top-Notch Top-Floor,8986314,Stephen,Greenpoint,Brooklyn,40.73198,-73.95446,Private room,NaN,30,1125,NaN
992,1702533,20250801203054,2025-08-02,BROOKLYN FAVORITE VACATION HOME 2,8335875,Mr. W,Park Slope,Brooklyn,40.68228,-73.97916,Entire home/apt,$330.00,30,30,NaN
993,1702781,20250801203054,2025-08-03,Nice and quiet private room,7614595,Maria,Astoria,Queens,40.76527,-73.92979,Private room,$47.00,30,180,NaN
994,1704560,20250801203054,2025-08-02,Beautiful/Spacious 1 Bedroom in LES,5693756,Daniel,Lower East Side,Manhattan,40.72110,-73.99157,Entire home/apt,NaN,30,30,NaN
995,1707850,20250801203054,2025-08-02,Cozy and quiet room PLG Brooklyn,9010960,Zulu & Andy,Prospect-Lefferts Gardens,Brooklyn,40.65897,-73.94957,Private room,$69.00,30,1125,NaN
996,1716640,20250801203054,2025-08-02,One Stop to TimesSQ Vacation Apt,9051298,Lydiah,Harlem,Manhattan,40.81212,-73.94651,Entire home/apt,NaN,30,100,NaN
997,1461965,20250801203054,2025-08-02,Private Bedroom WITH Living room and working s...,7833913,Vijaya,Crown Heights,Brooklyn,40.67480,-73.94521,Private room,$62.00,30,1125,NaN
998,1467106,20250801203054,2025-08-02,Bedford Stuyvesant Urban Hang Suite,7858405,Diavanna,Bedford-Stuyvesant,Brooklyn,40.69110,-73.94591,Private room,$200.00,30,1125,NaN
999,1467618,20250801203054,2025-08-02,Williamsburg- 1 min to Bedford Ave (L) Station,7860852,Matthew,Williamsburg,Brooklyn,40.71744,-73.95725,Entire home/apt,NaN,30,1125,NaN


In [22]:
!wget https://data.insideairbnb.com/belgium/vlg/antwerp/2025-06-25/data/listings.csv.gz -O data/raw/antwerp/2025-06-25/listings.csv.gz

--2025-09-09 01:34:32--  https://data.insideairbnb.com/belgium/vlg/antwerp/2025-06-25/data/listings.csv.gz
Resolving data.insideairbnb.com (data.insideairbnb.com)... 13.249.141.60, 13.249.141.25, 13.249.141.54, ...
Connecting to data.insideairbnb.com (data.insideairbnb.com)|13.249.141.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1254666 (1.2M) [application/x-gzip]
Saving to: ‘data/raw/antwerp/2025-06-25/listings.csv.gz’

data/raw/antwerp/20 100%[===================>]   1.20M  --.-KB/s    in 0.06s   

2025-09-09 01:34:32 (20.0 MB/s) - ‘data/raw/antwerp/2025-06-25/listings.csv.gz’ saved [1254666/1254666]



In [9]:
spark.createDataFrame(df_listings_pd).schema

StructType([StructField('id', LongType(), True), StructField('listing_url', StringType(), True), StructField('scrape_id', LongType(), True), StructField('last_scraped', StringType(), True), StructField('source', StringType(), True), StructField('name', StringType(), True), StructField('description', StringType(), True), StructField('neighborhood_overview', StringType(), True), StructField('picture_url', StringType(), True), StructField('host_id', LongType(), True), StructField('host_url', StringType(), True), StructField('host_name', StringType(), True), StructField('host_since', StringType(), True), StructField('host_location', StringType(), True), StructField('host_about', StringType(), True), StructField('host_response_time', StringType(), True), StructField('host_response_rate', StringType(), True), StructField('host_acceptance_rate', StringType(), True), StructField('host_is_superhost', StringType(), True), StructField('host_thumbnail_url', StringType(), True), StructField('host_p

In [10]:
schema = types.StructType([
    types.StructField('id', types.LongType(), True), 
    types.StructField('listing_url', types.StringType(), True), 
    types.StructField('scrape_id', types.LongType(), True), 
    types.StructField('last_scraped', types.StringType(), True), 
    types.StructField('source', types.StringType(), True), 
    types.StructField('name', types.StringType(), True), 
    types.StructField('description', types.StringType(), True), 
    types.StructField('neighborhood_overview', types.StringType(), True), 
    types.StructField('picture_url', types.StringType(), True), 
    types.StructField('host_id', types.LongType(), True), 
    types.StructField('host_url', types.StringType(), True), 
    types.StructField('host_name', types.StringType(), True), 
    types.StructField('host_since', types.StringType(), True), 
    types.StructField('host_location', types.StringType(), True), 
    types.StructField('host_about', types.StringType(), True), 
    types.StructField('host_response_time', types.StringType(), True), 
    types.StructField('host_response_rate', types.StringType(), True), 
    types.StructField('host_acceptance_rate', types.StringType(), True), 
    types.StructField('host_is_superhost', types.StringType(), True), 
    types.StructField('host_thumbnail_url', types.StringType(), True), 
    types.StructField('host_picture_url', types.StringType(), True), 
    types.StructField('host_neighbourhood', types.StringType(), True), 
    types.StructField('host_listings_count', types.LongType(), True), 
    types.StructField('host_total_listings_count', types.LongType(), True), 
    types.StructField('host_verifications', types.StringType(), True), 
    types.StructField('host_has_profile_pic', types.StringType(), True), 
    types.StructField('host_identity_verified', types.StringType(), True), 
    types.StructField('neighbourhood', types.StringType(), True), 
    types.StructField('neighbourhood_cleansed', types.StringType(), True), 
    types.StructField('neighbourhood_group_cleansed', types.StringType(), True), 
    types.StructField('latitude', types.DoubleType(), True), 
    types.StructField('longitude', types.DoubleType(), True), 
    types.StructField('property_type', types.StringType(), True), 
    types.StructField('room_type', types.StringType(), True), 
    types.StructField('accommodates', types.LongType(), True), 
    types.StructField('bathrooms', types.DoubleType(), True), 
    types.StructField('bathrooms_text', types.StringType(), True), 
    types.StructField('bedrooms', types.DoubleType(), True), 
    types.StructField('beds', types.DoubleType(), True), 
    types.StructField('amenities', types.StringType(), True), 
    types.StructField('price', types.StringType(), True), 
    types.StructField('minimum_nights', types.LongType(), True), 
    types.StructField('maximum_nights', types.LongType(), True), 
    types.StructField('minimum_minimum_nights', types.LongType(), True), 
    types.StructField('maximum_minimum_nights', types.LongType(), True), 
    types.StructField('minimum_maximum_nights', types.LongType(), True), 
    types.StructField('maximum_maximum_nights', types.LongType(), True), 
    types.StructField('minimum_nights_avg_ntm', types.DoubleType(), True), 
    types.StructField('maximum_nights_avg_ntm', types.DoubleType(), True), 
    types.StructField('calendar_updated', types.DoubleType(), True), 
    types.StructField('has_availability', types.StringType(), True), 
    types.StructField('availability_30', types.LongType(), True), 
    types.StructField('availability_60', types.LongType(), True), 
    types.StructField('availability_90', types.LongType(), True), 
    types.StructField('availability_365', types.LongType(), True), 
    types.StructField('calendar_last_scraped', types.StringType(), True), 
    types.StructField('number_of_reviews', types.LongType(), True), 
    types.StructField('number_of_reviews_ltm', types.LongType(), True), 
    types.StructField('number_of_reviews_l30d', types.LongType(), True), 
    types.StructField('availability_eoy', types.LongType(), True), 
    types.StructField('number_of_reviews_ly', types.LongType(), True), 
    types.StructField('estimated_occupancy_l365d', types.LongType(), True), 
    types.StructField('estimated_revenue_l365d', types.DoubleType(), True), 
    types.StructField('first_review', types.StringType(), True), 
    types.StructField('last_review', types.StringType(), True), 
    types.StructField('review_scores_rating', types.DoubleType(), True), 
    types.StructField('review_scores_accuracy', types.DoubleType(), True), 
    types.StructField('review_scores_cleanliness', types.DoubleType(), True), 
    types.StructField('review_scores_checkin', types.DoubleType(), True), 
    types.StructField('review_scores_communication', types.DoubleType(), True), 
    types.StructField('review_scores_location', types.DoubleType(), True), 
    types.StructField('review_scores_value', types.DoubleType(), True), 
    types.StructField('license', types.StringType(), True), 
    types.StructField('instant_bookable', types.StringType(), True), 
    types.StructField('calculated_host_listings_count', types.LongType(), True), 
    types.StructField('calculated_host_listings_count_entire_homes', types.LongType(), True), 
    types.StructField('calculated_host_listings_count_private_rooms', types.LongType(), True), 
    types.StructField('calculated_host_listings_count_shared_rooms', types.LongType(), True), 
    types.StructField('reviews_per_month', types.DoubleType(), True)
])

In [11]:
df_listings = spark.read \
    .option("header", "true") \
    .option("encoding", "utf-8") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .option("multiLine", "true") \
    .option("mode", "PERMISSIVE") \
    .schema(schema) \
    .csv("data/raw/antwerp/2025-06-25/listings.csv.gz")

In [12]:
listings_filtered_schema = types.StructType([
    types.StructField('id', types.LongType(), True), 
    types.StructField('scrape_id', types.LongType(), True), 
    types.StructField('last_scraped', types.StringType(), True), 
    types.StructField('name', types.StringType(), True), 
    types.StructField('host_id', types.LongType(), True), 
    types.StructField('host_name', types.StringType(), True), 
    types.StructField('neighbourhood_cleansed', types.StringType(), True), 
    types.StructField('neighbourhood_group_cleansed', types.StringType(), True), 
    types.StructField('latitude', types.DoubleType(), True), 
    types.StructField('longitude', types.DoubleType(), True), 
    types.StructField('room_type', types.StringType(), True), 
    types.StructField('price', types.StringType(), True), 
    types.StructField('minimum_nights', types.LongType(), True), 
    types.StructField('maximum_nights', types.LongType(), True),  
    types.StructField('license', types.StringType(), True)
])


In [13]:
special_casts = {
    "price": lambda col: F.regexp_replace(col, "[$,]", "").cast(types.DoubleType()),
    "last_scraped": lambda col: F.to_date(col, "yyyy-MM-dd")
}

In [14]:
for field in listings_filtered_schema.fields:
    col_name = field.name
    col_type = field.dataType

    if col_name in special_casts:
        df_listings = df_listings.withColumn(col_name, special_casts[col_name](F.col(col_name)))
    else:
        df_listings = df_listings.withColumn(col_name, F.col(col_name).cast(col_type))

In [15]:
df_listings.createOrReplaceTempView("listings")

25/09/09 10:44:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [17]:
spark.sql("""
select 
    id
from listings
sort by id asc
limit 20
""").show()

+-------+
|     id|
+-------+
|  50904|
| 345959|
| 366252|
| 522693|
| 603545|
| 772842|
| 778043|
| 813969|
| 890384|
| 891884|
| 891907|
| 959200|
| 991824|
|1050693|
|1123076|
|1172507|
|1184623|
|1227242|
|1328770|
|1808782|
+-------+



In [18]:
spark.sql("""
select 
    id,last_scraped,host_id,
    neighbourhood_cleansed,neighbourhood_group_cleansed,
    room_type,price,minimum_nights,license
from listings
limit 10
offset 990
""").show()

+-------+------------+-------+----------------------+----------------------------+---------------+-----+--------------+-------+
|     id|last_scraped|host_id|neighbourhood_cleansed|neighbourhood_group_cleansed|      room_type|price|minimum_nights|license|
+-------+------------+-------+----------------------+----------------------------+---------------+-----+--------------+-------+
|1696381|  2025-08-02|2717428|           East Harlem|                   Manhattan|   Private room| NULL|            30|   NULL|
|1702032|  2025-08-02|8986314|            Greenpoint|                    Brooklyn|   Private room| NULL|            30|   NULL|
|1702533|  2025-08-02|8335875|            Park Slope|                    Brooklyn|Entire home/apt|330.0|            30|   NULL|
|1702781|  2025-08-03|7614595|               Astoria|                      Queens|   Private room| 47.0|            30|   NULL|
|1704560|  2025-08-02|5693756|       Lower East Side|                   Manhattan|Entire home/apt| NULL|

In [19]:
spark.sql("""
select count(*)
from listings
""").show()

[Stage 3:>                                                          (0 + 1) / 1]

+--------+
|count(1)|
+--------+
|   36403|
+--------+



In [20]:
spark.sql("""
select room_type, count(*)
from listings
group by room_type
""").show()

[Stage 6:>                                                          (0 + 1) / 1]

+---------------+--------+
|      room_type|count(1)|
+---------------+--------+
|    Shared room|     228|
|     Hotel room|     378|
|Entire home/apt|   19328|
|   Private room|   16469|
+---------------+--------+



In [28]:
spark.sql("""
select license, count(*)
from listings
where 
group by license
order by count(*) desc
limit 100
""").show()

[Stage 25:>                                                         (0 + 1) / 1]

+------------------+--------+
|           license|count(1)|
+------------------+--------+
|              NULL|   30937|
|            Exempt|    3186|
|OSE-STRREG-0000437|       7|
|OSE-STRREG-0001957|       6|
|OSE-STRREG-0003000|       5|
|OSE-STRREG-0001246|       5|
|OSE-STRREG-0002440|       4|
|OSE-STRREG-0000207|       4|
|OSE-STRREG-0000670|       4|
|OSE-STRREG-0000299|       4|
|OSE-STRREG-0001191|       3|
|OSE-STRREG-0000027|       3|
|OSE-STRREG-0001804|       3|
|OSE-STRREG-0001970|       3|
|OSE-STRREG-0002619|       3|
|OSE-STRREG-0000315|       3|
|OSE-STRREG-0001177|       3|
|OSE-STRREG-0000549|       3|
|OSE-STRREG-0000075|       3|
|OSE-STRREG-0000587|       3|
+------------------+--------+
only showing top 20 rows



In [52]:
spark.sql("""
select 
    host_id,
    host_name,
    room_type,
    count(*) as total_listings
from listings
group by host_id, host_name, room_type
order by total_listings desc
limit 10
""").show()

[Stage 24:>                                                         (0 + 1) / 1]

+---------+-----------------+---------------+--------------+
|  host_id|        host_name|      room_type|total_listings|
+---------+-----------------+---------------+--------------+
|107434423|       Blueground|Entire home/apt|          1087|
|  3223938|           Eugene|   Private room|           598|
|446820235|LuxurybookingsFZE|Entire home/apt|           331|
| 19303369|           Hiroki|   Private room|           248|
|162280872|  Urban Furnished|Entire home/apt|           239|
|533234561|             Tina|   Private room|           227|
|200239515|            Shogo|   Private room|           222|
|204704622|           Momoyo|   Private room|           213|
| 51501835|         Jeniffer|Entire home/apt|           183|
| 35491667|              Nat|Entire home/apt|           175|
+---------+-----------------+---------------+--------------+



In [40]:
spark.sql("""
with cte as (
    select host_id, count(*) as total_listings
    from listings
    group by host_id
), less_then_ten_listings as (
    select cast(total_listings as string) total_listings, sum(total_listings)
    from cte
    where total_listings < 10
    group by total_listings
    order by total_listings
), remaining_listings as (
    select '10+', sum(total_listings)
    from cte
    where total_listings >= 10
)
select * from less_then_ten_listings
union all
select * from remaining_listings
""").show()

+--------------+-------------------+
|total_listings|sum(total_listings)|
+--------------+-------------------+
|             1|              18080|
|             2|               4080|
|             3|               1869|
|             4|               1128|
|             5|                680|
|             6|                612|
|             7|                364|
|             8|                440|
|             9|                432|
|           10+|               8718|
+--------------+-------------------+



In [53]:
spark.sql("""
select 
    minimum_nights,
    count(*) as total_listings
from listings
group by minimum_nights
order by total_listings desc
limit 10
""").show()

[Stage 27:>                                                         (0 + 1) / 1]

+--------------+--------------+
|minimum_nights|total_listings|
+--------------+--------------+
|            30|         28912|
|             1|          3000|
|            31|          1353|
|             2|           965|
|             3|           713|
|             4|           235|
|            90|           205|
|            60|           192|
|             5|           173|
|           180|            79|
+--------------+--------------+



In [55]:
spark.sql("""
with cte as (
    select 
        minimum_nights,
        count(*) as total_listings
    from listings
    group by minimum_nights
)
select sum(total_listings) as total_listings, 'short-term' as term
from cte
where minimum_nights < 30
union all
select sum(total_listings) as total_listings, 'long-term'
from cte
where minimum_nights >= 30
""").show()

[Stage 42:>                                                         (0 + 1) / 1]

+--------------+----------+
|total_listings|      term|
+--------------+----------+
|          5260|short-term|
|         31143| long-term|
+--------------+----------+



In [56]:
spark.sql("""
SELECT
    SUM(CASE WHEN minimum_nights < 30 THEN 1 ELSE 0 END) AS short_term_listings,
    SUM(CASE WHEN minimum_nights >= 30 THEN 1 ELSE 0 END) AS long_term_listings
FROM listings
""").show()

[Stage 52:>                                                         (0 + 1) / 1]

+-------------------+------------------+
|short_term_listings|long_term_listings|
+-------------------+------------------+
|               5260|             31143|
+-------------------+------------------+

